In [54]:
%load_ext autoreload
%autoreload 2
import markovify
from Bio import SeqIO
import numpy as np
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
def load_genome(filepath):
    """
    Load a genome sequence from a FASTA file.
    """
    for record in SeqIO.parse(filepath, "fasta"):
        return str(record.seq)

def generate_kmers(sequence, k):
    """
    Generate k-mers from a given sequence.
    """
    return [sequence[i:i+k] for i in range(len(sequence) - k + 1)]

# Load the genome (update the path to the actual file)
genome = load_genome("escherichia_coli_reference.fasta")
print(f"Genome loaded: {len(genome)} bases")

k = 50000  # Length of k-mers
kmers = generate_kmers(genome, k)

print(f"Number of {k}-mers: {len(kmers)}")

n_kmers = len(kmers)
unique_kmers = set(kmers)

kmer_dict = {kmer: idx for idx, kmer in enumerate(unique_kmers)}
reverse_kmer_dict = {idx: kmer for kmer, idx in kmer_dict.items()}
base_encoder = {"A": 0, "C": 1, "G": 2, "T": 3}
reverse_base_decoder = {0: "A", 1: "C", 2: "G", 3: "T"}

Genome loaded: 4641652 bases


In [ ]:
transition_matrix = np.zeros((len(unique_kmers), 4))

In [ ]:
transition_matrix.shape

In [ ]:
starting_state = kmer_dict[kmers[0]]
starting_state

In [ ]:
curr = kmers[0]
for i in range(n_kmers-1):
    prev = kmers[i] #Previous kmer
    curr = kmers[i+1][-1] #Next base
    i_idx = kmer_dict[prev]
    j_idx = base_encoder[curr]
    transition_matrix[i_idx, j_idx]+=1

In [ ]:
final_state = np.where(np.sum(transition_matrix, axis=1) == 0.)[0]
mask = np.ones(transition_matrix.shape[0], dtype=bool)
mask[final_state] = False

In [ ]:
if final_state.size != 0:
    transition_matrix[mask] = transition_matrix[mask] / np.sum(transition_matrix[mask], axis=1)[:, None]
else:
    transition_matrix = transition_matrix / np.sum(transition_matrix, axis=1)[:, None]

In [ ]:
transition_matrix

In [111]:
(np.sum(transition_matrix, axis=1)==0).sum()

np.int64(1)

In [112]:
np.isclose(np.sum(transition_matrix, 1), 1).sum()

np.int64(4578839)

In [113]:
def sample_sequence(transition_matrix, reverse_kmer_dict, base_decoder, starting_state, sequence_length, final_state = None):
    """
    Generate a random sequence from the Markov chain.
    
    Parameters:
    - transition_matrix: The normalized transition matrix (rows sum to 1).
    - reverse_kmer_dict: Dictionary to decode integer indices back to k-mers.
    - base_decoder: Dictionary mapping integers to bases (reverse of base_encoder).
    - starting_state: The index of the starting k-mer.
    - sequence_length: The desired length of the generated sequence.
    
    Returns:
    - A string representing the generated sequence.
    """
    current_state = starting_state
    kmer = reverse_kmer_dict[current_state]
    k = len(kmer)

    # Initialize a list for sequence construction
    sequence = list(kmer)

    cumulative_probs = np.cumsum(transition_matrix, axis=1)
    for _ in range(sequence_length - k):
        rand_val = np.random.rand()
        next_base_idx = np.searchsorted(cumulative_probs[current_state], rand_val)
        next_base = base_decoder[next_base_idx]

        # Append the sampled base to the sequence
        sequence.append(next_base)

        # Shift the k-mer to include the next base
        next_kmer = ''.join(sequence[-k:])
        current_state = kmer_dict[next_kmer]
        if next_base_idx == final_state:
            break

    # Convert the list of characters back to a string
    return ''.join(sequence)

In [114]:
sampled_sequence = sample_sequence(transition_matrix, reverse_kmer_dict, reverse_base_decoder, starting_state, len(genome), None)

In [115]:
print(sampled_sequence[:20])

AGCTTTTCATTCTGACTGCA


In [116]:
print(genome[:20])

AGCTTTTCATTCTGACTGCA


In [117]:
def save_to_fasta(sequence, output_path, sequence_id="sampled_sequence"):
    # Create a SeqRecord
    record = SeqRecord(Seq(sequence),
                       id=sequence_id,
                       description="Generated sampled sequence")
    with open(output_path, "w") as fasta_file:
        SeqIO.write(record, fasta_file, "fasta")

In [118]:
save_to_fasta(sampled_sequence, "sampled_sequence.fasta")

In [119]:
407520/len(genome)

0.08779632768678049